In [1]:
# import sys 

import pygame
from factory import factory
from typing import List, Tuple

class Building:
    '''A class representing a building with floors and elevators.

    Attributes:
        last_time (float): The time in seconds of the last update.
        num_floors (int): The number of floors in the building.
        floor_width (int): The width of each floor in pixels.
        floor_height (int): The height of each floor in pixels.
        elevators (List[Elevator]): A list of Elevator objects in the building.
        floors (List[Floor]): A list of Floor objects in the building.
    '''

    def __init__(self, position: int, num_floors: int, num_elevators: int, floor_width: int, floor_height: int, height_screen: int):
        '''Initialize a Building object.
        
        Args:
            position (int): The x-coordinate position of the building.
            num_floors (int): The number of floors in the building.
            num_elevators (int): The number of elevators in the building.
            floor_width (int): The width of each floor in pixels.
            floor_height (int): The height of each floor in pixels.
            height_screen (int): The height of the screen in pixels.
        '''
        self.last_time: float = pygame.time.get_ticks() / 1000  # Store the time in seconds
        self.num_floors: int = num_floors
        self.floor_width: int = floor_width
        self.floor_height: int = floor_height
        self.elevators: List[Elevator] = [factory("elevator", i, floor_height, floor_width / 2, height_screen, floor_width * (i / 2 + 1) + position) for i in range(num_elevators)]
        self.floors: List[Floor] = [factory("floor", i, position, height_screen - (i + 1) * floor_height, floor_width, floor_height) for i in range(num_floors + 1)]

    def draw(self, surface: pygame.Surface) -> None:
        '''Draw the building and its components on the given surface.'''
        for floor in self.floors:
            floor.draw(surface)
        for elv in self.elevators:
            elv.draw(surface)

    def handle_events(self, mouse_pos: Tuple[int, int]) -> None:
        '''Handle mouse events for the building.'''
        for floor in self.floors:
            check = floor.handle_events(mouse_pos)
            if check == floor.number:
                min_time = float('inf')
                min_elv = 0
                for elv in self.elevators:
                    time_elv = elv.calculate_time(check)
                    if elv.target_floor == check:
                        return
                    if time_elv < min_time:
                        min_time = time_elv
                        min_elv = elv.number

                self.elevators[min_elv].add_to_queue(check)
                self.floors[check].increment_timer(min_time)
                return
                        
    def process_elevator_movement(self) -> None:
        '''Process the movement of elevators.'''
        current_time = pygame.time.get_ticks() / 1000  # in seconds
        for elv in self.elevators:
            tarrget_floor = elv.target_floor
            height_floor = self.floors[tarrget_floor].get_rect().top
            floor = elv.process_movement(height_floor, current_time, self.last_time)
        for floor in self.floors:
            floor.timer(current_time, self.last_time)
        self.last_time = current_time



from enum import Enum
import pygame
import queue

class Direction(Enum):
    '''An enumeration representing elevator directions.'''
    UP = "up"
    DOWN = "down"
    PLACE = "place"

class Elevator:
    '''A class representing an elevator in a building.

    Attributes:
        image (pygame.Surface): The image of the elevator.
        ring (None): The sound of the elevator.
        number (int): The number of the elevator.
        floor (int): The current floor of the elevator.
        location (Tuple[int, int]): The (x, y) coordinates of the elevator's location.
        target_floor (int): The target floor of the elevator.
        floor_height (int): The height of each floor in pixels.
        elevator_speed (float): The speed of the elevator in floors per second.
        time_elapsed (float): The time elapsed since the last movement.
        stay_time (float): The time the elevator stays idle on a floor.
        queue (queue.Queue): The queue of floors the elevator needs to visit.
        last_floor (int): The last floor the elevator visited.
        travel_direction (Direction): The direction of the elevator's travel.
        stay (bool): A flag indicating if the elevator is staying idle.
    '''

    def __init__(self, number: int, height: int, width: int, height_screen: int, x: int):
        '''Initialize an Elevator object.

        Args:
            number (int): The number of the elevator.
            height (int): The height of the elevator image.
            width (int): The width of the elevator image.
            height_screen (int): The height of the screen.
            x (int): The initial x-coordinate position of the elevator.
        '''
        self.image: pygame.Surface = pygame.transform.scale(pygame.image.load("elv.png"), (width, height))
        self.ring = pygame.mixer.music.load("ding.mp3")
        self.number: int = number
        self.floor: int = 0
        self.location: Tuple[int, int] = (x, height_screen - self.image.get_rect().height)
        self.target_floor: int = 0
        self.floor_height: int = height
        self.elevator_speed: float = 0.5  # Floors per second
        self.time_elapsed: float = 0
        self.stay_time: float = 0
        self.queue: queue.Queue = queue.Queue()
        self.last_floor: int = 0
        self.travel_direction: Direction = Direction.PLACE
        self.stay: bool = False

    def draw(self, surface: pygame.Surface) -> None:
        '''Draw the elevator on the given surface.'''
        surface.blit(self.image, self.location)

    def update_position(self, current_time: float, last_time: float, height_floor: int) -> int:
        '''Update the position of the elevator based on the current time and floor height.

        Args:
            current_time (float): The current time in seconds.
            last_time (float): The time of the last update in seconds.
            height_floor (int): The height of the target floor.

        Returns:
            int: 1 if the elevator reaches the target floor, 0 otherwise.
        '''
        distance_to_move: float = (current_time - last_time) * self.floor_height / self.elevator_speed
        if self.travel_direction in [Direction.UP, Direction.DOWN] and self.location[1] != height_floor:
            direction: int = 1 if self.travel_direction == Direction.UP else -1
            self.location = (self.location[0], self.location[1] + direction * distance_to_move)
        else:
            self.travel_direction = Direction.PLACE
            self.stay = True
            return 1
        return 0

    def adjust_stay_time(self, current_time: float, last_time: float) -> None:
        '''Adjust the stay time of the elevator.

        Args:
            current_time (float): The current time in seconds.
            last_time (float): The time of the last update in seconds.
        '''
        self.stay_time += current_time - last_time
        if self.stay_time >= 2:
            self.stay = False
            self.stay_time = 0

    def process_movement(self, height_floor: int, current_time: float, last_time: float) -> int:
        '''Process the movement of the elevator.

        Args:
            height_floor (int): The height of the target floor.
            current_time (float): The current time in seconds.
            last_time (float): The time of the last update in seconds.

        Returns:
            int: The target floor if reached, -1 otherwise.
        '''
        if self.time_elapsed > 0:
            self.add_time(last_time - current_time)
        else:
            self.time_elapsed = 0
        if self.stay:
            self.adjust_stay_time(current_time, last_time)
        elif self.travel_direction != Direction.PLACE:
            if self.update_position(current_time, last_time, height_floor):
                pygame.mixer.music.play()
                return self.target_floor
        elif not self.queue.empty():
            self.floor = self.target_floor
            self.target_floor = self.queue.get()
            self.travel_direction = Direction.UP if self.floor < self.target_floor else Direction.DOWN
        return -1

    def add_to_queue(self, number: int) -> None:
        '''Add a floor to the elevator's queue.

        Args:
            number (int): The floor number to add to the queue.
        '''
        self.queue.put(number)
        time_to_add: float = abs(number - self.last_floor) * self.elevator_speed + 2
        self.last_floor = number
        self.add_time(time_to_add)

    def add_time(self, number: float) -> None:
        '''Add time to the elevator's time elapsed.

        Args:
            number (float): The time to add in seconds.
        '''
        self.time_elapsed += number

    def calculate_time(self, floor: int) -> float:
        '''Calculate the time required to reach a floor.

        Args:
            floor (int): The target floor.

        Returns:
            float: The calculated time in seconds.
        '''
        return self.time_elapsed + abs(floor - self.last_floor) / 2


import pygame
from typing import Tuple

class Floor:
    '''A class representing a floor in a building.

    Attributes:
        number (int): The floor number.
        rect (pygame.Rect): The rectangle representing the floor's position and size.
        color (Tuple[int, int, int]): The color of the floor.
        brick_color (Tuple[int, int, int]): The color of the bricks on the floor.
        black_line_color (Tuple[int, int, int]): The color of the black line at the bottom of the floor.
        height_black_line (int): The height of the black line at the bottom of the floor.
        font (pygame.font.Font): The font used for rendering text.
        number_color (Tuple[int, int, int]): The color of the floor number.
        time (float): The time remaining on the floor's timer.
    '''

    def __init__(self, number: int, x: int, y: int, width: int, height: int) -> None:
        '''Initialize a Floor object.

        Args:
            number (int): The floor number.
            x (int): The x-coordinate of the top-left corner of the floor.
            y (int): The y-coordinate of the top-left corner of the floor.
            width (int): The width of the floor.
            height (int): The height of the floor.
        '''
        self.number: int = number
        self.rect: pygame.Rect = pygame.Rect(x, y, width, height)
        self.color: Tuple[int, int, int] = (192, 192, 192)
        self.brick_color: Tuple[int, int, int] = (255, 0, 0)
        self.black_line_color: Tuple[int, int, int] = (0, 0, 0)
        self.height_black_line: int = 4
        self.font: pygame.font.Font = pygame.font.SysFont(None, 24)
        self.number_color: Tuple[int, int, int] = (0, 0, 0)
        self.time: float = 0

    def draw(self, surface: pygame.Surface) -> None:
        '''Draw the floor on the given surface.'''
        self.draw_bricks(surface)
        self.draw_black_line(surface)
        self.draw_number(surface)
        if self.time > 0:
            self.draw_timer(surface)

    def draw_bricks(self, surface: pygame.Surface) -> None:
        '''Draw the bricks on the floor.'''
        pygame.draw.rect(surface, self.color, self.rect)
        brick_width: int = 4
        brick_height: int = 2
        spacing: int = 1
        for row in range(0, self.rect.height - brick_height, brick_height + spacing):
            col_start: int = int(brick_width / 2) if row % 2 == 0 else 0
            for col in range(col_start, self.rect.width - brick_width, brick_width + spacing):
                brick_rect: pygame.Rect = pygame.Rect(self.rect.left + col, self.rect.top + row, brick_width, brick_height)
                pygame.draw.rect(surface, self.brick_color, brick_rect)

    def draw_black_line(self, surface: pygame.Surface) -> None:
        '''Draw the black line at the bottom of the floor.'''
        line_rect: pygame.Rect = pygame.Rect(self.rect.left, self.rect.bottom - self.height_black_line, 
                                              self.rect.width, self.height_black_line)
        pygame.draw.rect(surface, self.black_line_color, line_rect)

    def draw_number(self, surface: pygame.Surface) -> None:
        '''Draw the floor number on the floor.'''
        if self.time > 0:
            color: Tuple[int, int, int] = (0, 255, 0)
            position: Tuple[int, int] = self.rect.left + 10, self.rect.top + 8
        else:
            color: Tuple[int, int, int] = (0, 0, 0)
            position: Tuple[int, int] = self.rect.left + 10, self.rect.top + 8
        pygame.draw.rect(surface, (192, 192, 192), (position[0], position[1], 20, 16))
        number_text: pygame.Surface = self.font.render(str(self.number), True, color)
        text_rect: pygame.Rect = number_text.get_rect(topleft=(position[0], position[1]))
        surface.blit(number_text, text_rect)

    def draw_timer(self, surface: pygame.Surface) -> None:
        '''Draw the timer text on the floor.'''
        timer_text: pygame.Surface = self.font.render(f"{self.time:.1f}", True, (255, 255, 255))
        timer_rect: pygame.Rect = timer_text.get_rect()
        timer_rect.left: int = self.rect.right + 5
        timer_rect.centery: int = self.rect.centery
        surface.blit(timer_text, timer_rect)

    def handle_events(self, mouse_pos: Tuple[int, int]) -> int:
        '''Handle mouse events on the floor.

        Args:
            mouse_pos (Tuple[int, int]): The mouse position.

        Returns:
            int: The floor number if clicked and the time is not running, otherwise -1.
        '''
        if self.rect.collidepoint(mouse_pos) and self.time <= 0:
            return self.number
        return -1

    def get_rect(self) -> pygame.Rect:
        '''Get the rectangle representing the floor's position and size.

        Returns:
            pygame.Rect: The rectangle representing the floor.
        '''
        return self.rect

    def timer(self, current_time: float, last_time: float) -> None:
        '''Update the floor timer.

        Args:
            current_time (float): The current time in seconds.
            last_time (float): The time of the last update in seconds.
        '''
        if self.time > 0:
            self.time -= current_time - last_time
        else:
            self.time = 0

    def increment_timer(self, time_to_add: float) -> None:
        '''Increment the floor timer.

        Args:
            time_to_add (float): The time to add to the timer in seconds.
        '''
        if self.time < 0:
            self.time = time_to_add
        else:
            self.time += time_to_add



from typing import Type, Any, Tuple, Dict, Union
from building import Building
from floor import Floor
from elevator import Elevator

def factory(object_type: str, *args: Any, **kwargs: Any) -> Union[Type[Building], Type[Floor], Type[Elevator], None]:
    '''Factory function to create objects of different types.
    It is made to use the "factory" pattern design.

    Args:
        object_type (str): The type of object to create.
        *args: Positional arguments to pass to the object constructor.
        **kwargs: Keyword arguments to pass to the object constructor.

    Returns:
        Union[Type[Building], Type[Floor], Type[Elevator], None]: The created object, or None if object_type is invalid.
    '''
    type_dict: Dict[str, Type[Any]] = {
        "building": Building,
        "floor": Floor,
        "elevator": Elevator
    }
    return type_dict[object_type](*args, **kwargs) if object_type in type_dict else None



# building_simulation.py
import pygame
import sys
from typing import List, Tuple

from building import Building
from factory import factory

class BuildingSimulation:
    def __init__(self, width_screen: int, height_screen: int, buildings_info: List[Tuple[int, int]]):
        '''Initialize the BuildingSimulation class.

        Args:
            width_screen (int): The width of the screen.
            height_screen (int): The height of the screen.
            buildings_info (List[Tuple[int, int]]): A list of tuples containing information about buildings,
                                                      where each tuple consists of the number of floors and
                                                      the number of elevators for each building.
        '''
        self.width_screen = width_screen
        self.height_screen = height_screen
        self.buildings_info = buildings_info
        self.screen = None
        self.buildings: List[Building] = []

    def initialize_pygame(self) -> None:
        '''Initialize Pygame and set the display caption.'''
        pygame.init()
        self.screen = pygame.display.set_mode((self.width_screen, self.height_screen))
        pygame.display.set_caption("Building Floors")

    def setup_buildings(self) -> None:
        '''Set up buildings based on the provided information.'''
        total_elevators = sum(info[1] / 2 + 1 for info in self.buildings_info)
        max_floors = max(info[0] for info in self.buildings_info) + 1
        count = 0
        for info in self.buildings_info:
            position = self.width_screen / total_elevators * count + 10
            building = factory("building", position, info[0], info[1], (self.width_screen - 20) / total_elevators,
                               (self.height_screen - 10) / max_floors, self.height_screen)
            self.buildings.append(building)
            count += info[1] / 2 + 1

    def handle_events(self) -> bool:
        '''Handle Pygame events, including quitting the game and mouse clicks.'''
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return False
            elif event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
                self.handle_mouse_click(pygame.mouse.get_pos())
        return True

    def handle_mouse_click(self, mouse_pos: Tuple[int, int]) -> None:
        '''Handle mouse clicks on the buildings.'''
        [building.handle_events(mouse_pos) for building in self.buildings]

    def draw_screen(self) -> None:
        '''Draw the screen by filling it with white color and drawing the buildings.'''
        self.screen.fill((255, 255, 255))
        [building.draw(self.screen) or building.process_elevator_movement() for building in self.buildings]
        pygame.display.flip()

    def run(self) -> None:
        '''Run the main loop for the building simulation.'''
        self.initialize_pygame()
        self.setup_buildings()
        clock = pygame.time.Clock()
        running = True
        while running:
            running = self.handle_events()
            self.draw_screen()
            clock.tick(60)
        pygame.quit()
        sys.exit()

# main.py

# from building_simulation import BuildingSimulation

def main() -> None:
    '''Main function to initialize and run the building simulation.'''
    width_screen: int = 1700
    height_screen: int = 900
    buildings_info: List[Tuple[int, int]] = [
        [15, 3],  # building1: 15 floors, 3 elevators
        [10, 2],  # building2: 10 floors, 2 elevators
        [20, 4]   # building3: 20 floors, 4 elevators
    ]
    simulation: BuildingSimulation = BuildingSimulation(width_screen, height_screen, buildings_info)
    simulation.run()

if __name__ == "__main__":
    main()


pygame 2.1.2 (SDL 2.0.16, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


ImportError: cannot import name 'factory' from partially initialized module 'factory' (most likely due to a circular import) (/home/mefathim/Documents/Mefathim/Ness/my_elevator/factory.py)

In [1]:
import pygame
import queue
import sys
from enum import Enum

class Building:
    def __init__(self, position, num_floors, num_elevators, floor_width, floor_height, height_screen):        
        from factory import factory
        self.last_time = pygame.time.get_ticks() / 1000  # Store the time in seconds
        self.num_floors = num_floors
        self.floor_width = floor_width
        self.floor_height = floor_height
        self.elevators = [factory("elevator", i, floor_height, floor_width / 2, height_screen, floor_width * (i / 2 + 1) + position) for i in range(num_elevators)]
        self.floors = [factory("floor", i, position, height_screen - (i + 1) * floor_height, floor_width, floor_height) for i in range(num_floors + 1)]

    def draw(self, surface):
        for floor in self.floors:
            floor.draw(surface)
        for elv in self.elevators:
            elv.draw(surface)

    def handle_events(self, mouse_pos):
        for floor in self.floors:
            check = floor.handle_events(mouse_pos)
            if check == floor.number:
                min_time = float('inf')
                min_elv = 0
                for elv in self.elevators:
                    time_elv = elv.calculate_time(check)
                    if elv.target_floor == check:
                        return
                    if time_elv < min_time:
                        min_time = time_elv
                        min_elv = elv.number

                self.elevators[min_elv].add_to_queue(check)
                self.floors[check].increment_timer(min_time)
                return
                        
    def process_elevator_movement(self):
        current_time = pygame.time.get_ticks() / 1000  # in seconds
        for elv in self.elevators:
            tarrget_floor = elv.target_floor
            height_floor = self.floors[tarrget_floor].get_rect().top
            floor = elv.process_movement(height_floor, current_time, self.last_time)
        for floor in self.floors:
            floor.timer(current_time, self.last_time)
        self.last_time = current_time
        
class Direction(Enum):
    UP = "up"
    DOWN = "down"
    PLACE = "place"

class Elevator:
    def __init__(self, number, height, width, height_screen, x):
        self.image = pygame.transform.scale(pygame.image.load("elv.png"), (width, height))
        self.ring = pygame.mixer.music.load("ding.mp3")
        self.number = number
        self.floor = 0
        self.location = (x, height_screen - self.image.get_rect().height)
        self.target_floor = 0
        self.floor_height = height
        self.elevator_speed = 0.5  # Floors per second
        self.time_elapsed = 0
        self.stay_time = 0
        self.queue = queue.Queue()
        self.last_floor = 0
        self.travel_direction = Direction.PLACE
        self.stay = False

    def draw(self, surface):
        surface.blit(self.image, self.location)

    def update_position(self, current_time, last_time, height_floor):
        distance_to_move = (current_time - last_time) * self.floor_height / self.elevator_speed
        if self.travel_direction in [Direction.UP, Direction.DOWN] and self.location[1] != height_floor:
            direction = 1 if self.travel_direction == Direction.UP else -1
            self.location = (self.location[0], self.location[1] + direction * distance_to_move)
        else:
            self.travel_direction = Direction.PLACE
            self.stay = True
            return 1
        return 0

    def adjust_stay_time(self, current_time, last_time):
        self.stay_time += current_time - last_time
        if self.stay_time >= 2:
            self.stay = False
            self.stay_time = 0

    def process_movement(self, height_floor, current_time, last_time):
        if self.time_elapsed > 0:
            self.add_time(last_time - current_time)
        else:
            self.time_elapsed = 0
        if self.stay:
            self.adjust_stay_time(current_time, last_time)
        elif self.travel_direction != Direction.PLACE:
            if self.update_position(current_time, last_time, height_floor):
                pygame.mixer.music.play()
                return self.target_floor
        elif not self.queue.empty():
            self.floor = self.target_floor
            self.target_floor = self.queue.get()
            self.travel_direction = Direction.UP if self.floor < self.target_floor else Direction.DOWN
        return -1

    def add_to_queue(self, number):
        self.queue.put(number)
        time_to_add = abs(number - self.last_floor) * self.elevator_speed + 2
        self.last_floor = number
        self.add_time(time_to_add)

    def add_time(self, number):
        self.time_elapsed += number

    def calculate_time(self, floor):
        return self.time_elapsed + abs(floor - self.last_floor) / 2
    
    
class Floor:
    def __init__(self, number, x, y, width, height):
        self.number = number
        self.rect = pygame.Rect(x, y, width, height)
        self.color = (192, 192, 192)
        self.brick_color = (255, 0, 0)
        self.black_line_color = (0, 0, 0)
        self.height_black_line = 4
        self.font = pygame.font.SysFont(None, 24)
        self.number_color = (0, 0, 0)
        self.time = 0

    def draw(self, surface):
        self.draw_bricks(surface)
        self.draw_black_line(surface)
        self.draw_number(surface)
        if self.time > 0:
            self.draw_timer(surface)

    def draw_bricks(self, surface):
        pygame.draw.rect(surface, self.color, self.rect)
        brick_width = 4
        brick_height = 2
        spacing = 1
        for row in range(0, self.rect.height - brick_height, brick_height + spacing):
            col_start = int(brick_width / 2) if row % 2 == 0 else 0
            for col in range(col_start, self.rect.width - brick_width, brick_width + spacing):
                brick_rect = pygame.Rect(self.rect.left + col, self.rect.top + row, brick_width, brick_height)
                pygame.draw.rect(surface, self.brick_color, brick_rect)

    def draw_black_line(self, surface):
        line_rect = pygame.Rect(self.rect.left, self.rect.bottom - self.height_black_line, 
                                self.rect.width, self.height_black_line)
        pygame.draw.rect(surface, self.black_line_color, line_rect)

    def draw_number(self, surface):
        if self.time > 0:
            color = (0, 255, 0)
            position = self.rect.left + 10, self.rect.top + 8
        else:
            color = (0, 0, 0)
            position = self.rect.left + 10, self.rect.top + 8
        pygame.draw.rect(surface, (192, 192, 192), (position[0], position[1], 20, 16))
        number_text = self.font.render(str(self.number), True, color)
        text_rect = number_text.get_rect(topleft=(position[0], position[1]))
        surface.blit(number_text, text_rect)

    def draw_timer(self, surface):
        timer_text = self.font.render(f"{self.time:.1f}", True, (255, 255, 255))
        timer_rect = timer_text.get_rect()
        timer_rect.left = self.rect.right + 5
        timer_rect.centery = self.rect.centery
        surface.blit(timer_text, timer_rect)

    def handle_events(self, mouse_pos):
        if self.rect.collidepoint(mouse_pos) and self.time <= 0:
            return self.number
        return -1

    def get_rect(self):
        return self.rect

    def timer(self, current_time, last_time):
        if self.time > 0:
            self.time -= current_time - last_time
        else:
            self.time = 0

    def increment_timer(self, time_to_add):
        if self.time < 0:
            self.time = time_to_add
        else:
            self.time += time_to_add
            
def factory(object_type, *args, **kwargs):
    type_dict = {
        "building": Building,
        "floor": Floor,
        "elevator": Elevator
    }
    return type_dict[object_type](*args, **kwargs) if object_type in type_dict else None

class BuildingSimulation:
    def __init__(self, width_screen, height_screen, buildings_info):
        self.width_screen = width_screen
        self.height_screen = height_screen
        self.buildings_info = buildings_info
        self.screen = None
        self.buildings = []

    def initialize_pygame(self):
        pygame.init()
        self.screen = pygame.display.set_mode((self.width_screen, self.height_screen))
        pygame.display.set_caption("Building Floors")

    def setup_buildings(self):
        total_elevators = sum(info[1] / 2 + 1 for info in self.buildings_info)
        max_floors = max(info[0] for info in self.buildings_info) + 1
        count = 0
        for info in self.buildings_info:
            position = self.width_screen / total_elevators * count + 10
            building = factory("building", position, info[0], info[1], (self.width_screen - 20) / total_elevators,
                               (self.height_screen - 10) / max_floors, self.height_screen)
            self.buildings.append(building)
            count += info[1] / 2 + 1

    def handle_events(self):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return False
            elif event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
                self.handle_mouse_click(pygame.mouse.get_pos())
        return True


    def handle_mouse_click(self, mouse_pos):
        [building.handle_events(mouse_pos) for building in self.buildings]

    def draw_screen(self):
        self.screen.fill((255, 255, 255))
        [building.draw(self.screen) or building.process_elevator_movement() for building in self.buildings]
        pygame.display.flip()

    def run(self):
        self.initialize_pygame()
        self.setup_buildings()
        clock = pygame.time.Clock()
        running = True
        while running:
            running = self.handle_events()
            self.draw_screen()
            clock.tick(60)
        pygame.quit()
        sys.exit()
        
def main():
    width_screen = 1700
    height_screen = 900
    buildings_info = [
        [15, 3],  # building1: 15 floors, 3 elevators
        [10, 2],  # building2: 10 floors, 2 elevators
        [20, 4]   # building3: 20 floors, 4 elevators
    ]
    simulation = BuildingSimulation(width_screen, height_screen, buildings_info)
    simulation.run()

if __name__ == "__main__":
    main() 

pygame 2.1.2 (SDL 2.0.16, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
import pygame
import queue
from enum import Enum

class Direction(Enum):
    UP = "up"
    DOWN = "down"
    PLACE = "place"

class Elevator:
    def __init__(self, number, height, width, height_screen, x):
        self.image = pygame.image.load("elv.png")
        self.image = pygame.transform.scale(self.image, (width, height))
        self.ring = pygame.mixer.music.load("ding.mp3")
        self.number = number
        self.floor = 0
        self.location = (x, height_screen - self.image.get_rect().height)
        self.target_floor = 0
        self.floor_height = height
        self.elevator_speed = 0.5  # Floors per second
        self.time_elapsed = 0
        self.stay_time = 0
        self.queue = queue.Queue()
        self.last_floor = 0
        self.travel_direction = Direction.PLACE
        self.stay = False

    def draw(self, surface):
        surface.blit(self.image, self.location)

    def update_position(self, current_time, last_time, height_floor):
        distance_to_move = (current_time - last_time) * self.floor_height / self.elevator_speed
        if self.travel_direction == Direction.UP and self.location[1] > height_floor:
            self.location = (self.location[0], self.location[1] - distance_to_move)
        elif self.travel_direction == Direction.DOWN and self.location[1] < height_floor:
            self.location = (self.location[0], self.location[1] + distance_to_move)
        else:
            self.travel_direction = Direction.PLACE
            self.stay = True
            return 1
        return 0

    def adjust_stay_time(self, current_time, last_time):
        self.stay_time += current_time - last_time
        if self.stay_time >= 2:
            self.stay = False
            self.stay_time = 0

    def process_movement(self, height_floor, current_time, last_time):
        if self.time_elapsed > 0:
            self.add_time(last_time - current_time)
        else:
            self.time_elapsed = 0
        if self.stay:
            self.adjust_stay_time(current_time, last_time)
        elif self.travel_direction != Direction.PLACE:
            if self.update_position(current_time, last_time, height_floor):
                pygame.mixer.music.play()
                return self.target_floor
        elif not self.queue.empty():
            self.floor = self.target_floor
            self.target_floor = self.queue.get()
            if self.floor < self.target_floor:
                self.travel_direction = Direction.UP
            else:
                self.travel_direction = Direction.DOWN
        return -1

    def add_to_queue(self, number):
        self.queue.put(number)
        time_to_add = abs(number - self.last_floor) * self.elevator_speed + 2
        self.last_floor = number
        self.add_time(time_to_add)

    def add_time(self, number):
        self.time_elapsed += number

    def calculate_time(self, floor):
        return self.time_elapsed + abs(floor - self.last_floor) / 2

class Floor:
    def __init__(self, number, x, y, width, height):
        self.number = number
        self.rect = pygame.Rect(x, y, width, height)
        self.color = (192, 192, 192)
        self.brick_color = (255, 0, 0)
        self.black_line_color = (0, 0, 0)
        self.height_black_line = 4
        self.font = pygame.font.SysFont(None, 24)
        self.number_color = (0, 0, 0)
        self.time = 0

    def draw(self, surface):
        self.draw_bricks(surface)
        self.draw_black_line(surface)
        self.draw_number(surface)
        if self.time > 0:
            self.draw_timer(surface)

    def draw_bricks(self, surface):
        pygame.draw.rect(surface, self.color, self.rect)
        brick_width = 4
        brick_height = 2
        spacing = 1
        i = 0
        for row in range(0, self.rect.height - brick_height, brick_height + spacing):
            # Adjust col start based on the row index to create a staggered brick pattern
            col_start = int(brick_width / 2) if i % 2 == 0 else 0
            for col in range(col_start, self.rect.width - brick_width, brick_width + spacing):
                brick_rect = pygame.Rect(self.rect.left + col, self.rect.top + row, brick_width, brick_height)
                pygame.draw.rect(surface, self.brick_color, brick_rect)
            i += 1

    def draw_black_line(self, surface):
        line_rect = pygame.Rect(self.rect.left, self.rect.bottom - self.height_black_line, 
                                self.rect.width, self.height_black_line)
        pygame.draw.rect(surface, self.black_line_color, line_rect)

    def draw_number(self, surface):
        # Calculate position and color settings
        number_rect_width, number_rect_height = 20, 16
        number_rect_position, self.number_color = self.calculate_number_position(number_rect_width, number_rect_height)

        # Draw the background rectangle for the number
        pygame.draw.rect(surface, (192, 192, 192), number_rect_position)
        
        # Render the number text and blit to the surface
        number_text = self.font.render(str(self.number), True, self.number_color)
        text_rect = number_text.get_rect(center=number_rect_position.center)
        surface.blit(number_text, text_rect)

    def calculate_number_position(self, width, height):
        # Determine color and position based on the timer state
        if self.time > 0:
            color = (0, 255, 0)
            position = pygame.Rect(self.rect.right - width - 10,
                                   self.rect.centery - (height / 2),
                                   width, height)
        else:
            color = (0, 0, 0)
            position = pygame.Rect(self.rect.centerx - (width / 2),
                                   self.rect.centery - (height / 2),
                                   width, height)
        return position, color
    
    def draw_timer(self, surface):
        timer_text = self.font.render(f"{self.time:.1f}", True, (255, 255, 255))
        timer_rect = timer_text.get_rect(center=(self.rect.left + 20, self.rect.centery))
        surface.blit(timer_text, timer_rect)

    def handle_events(self, mouse_pos):
        if self.rect.collidepoint(mouse_pos) and self.time <= 0:
            return self.number
        return -1

    def get_rect(self):
        return self.rect

    def timer(self, current_time, last_time):
        if self.time > 0:
            self.time -= current_time-last_time
        else:
            self.time = 0

    def increment_timer(self, time_to_add):
        if self.time < 0:
            self.time = time_to_add
        else:
            self.time += time_to_add

class Building:
    def __init__(self, position, num_floors, num_elevators, floor_width, floor_height, height_screen):        
        self.last_time = pygame.time.get_ticks() / 1000 #in second
        self.num_floors = num_floors
        self.floor_width = floor_width
        self.floor_height = floor_height
        self.elevators = []
        for i in range(num_elevators):
            elv = Elevator(i, floor_height, floor_width/2 , height_screen, floor_width * (i/2 +1) + position)
            self.elevators.append(elv)
        self.floors = []
        for i in range(num_floors+1):
            floor = Floor(i, position, height_screen - (i + 1) * floor_height, floor_width, floor_height)
            self.floors.append(floor)

    def draw(self, surface):
        for floor in self.floors:
            floor.draw(surface)
        for elv in self.elevators:
            elv.draw(surface)

    def handle_events(self, mouse_pos):
        for floor in self.floors:
            check = floor.handle_events(mouse_pos)
            if check == floor.number:
                min_time = float('inf')
                min_elv = 0
                for elv in self.elevators:
                    time_elv = elv.calculate_time(check)
                    if elv.target_floor == check:
                        return
                    if time_elv < min_time:
                        min_time = time_elv
                        min_elv = elv.number

                self.elevators[min_elv].add_to_queue(check)
                self.floors[check].increment_timer(min_time)
                return
                        
    def process_elevator_movement(self):
        current_time = pygame.time.get_ticks() / 1000 #in second
        for elv in self.elevators:
            tarrget_floor = elv.target_floor
            height_floor = self.floors[tarrget_floor].get_rect().top
            floor = elv.process_movement(height_floor, current_time, self.last_time)
        for floor in self.floors:
            floor.timer(current_time, self.last_time)
        self.last_time = current_time

class Street:
    def __init__(self, buidings_info, height_screen, width_screen):
        self.buildings = []
        sum = 0
        max_floor = 0
        for i in buidings_info:
            sum += i[1]/2 + 1
            if i[0] > max_floor:
                max_floor = i[0]
        max_floor += 1
        count = 0
        for i in buidings_info:
            position = width_screen/sum * count + 10
            buiding = Building(position, i[0], i[1], (width_screen - 20)/sum, (height_screen-10)/max_floor, height_screen)
            self.buildings.append(buiding)
            count += i[1]/2 + 1

    def draw(self, surface):
        for building in self.buildings:
            building.draw(surface)
            building.process_elevator_movement()
            
    def handle_events(self, mouse_pos):
        for building in self.buildings:
            building.handle_events(mouse_pos)

def main():
    height_screen = 700
    width_screen = 1200
    buildings_info = [
        [15, 3],  # building1: 15 floors, 3 elevators
        [10, 2],  # building2: 10 floors, 2 elevators
        [20, 4]   # building3: 20 floors, 4 elevators
    ]
    
    pygame.init()
    screen = pygame.display.set_mode((width_screen, height_screen))
    pygame.display.set_caption("Building Floors")
    street = Street(buildings_info, height_screen, width_screen)
    clock = pygame.time.Clock()

    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
                street.handle_events(pygame.mouse.get_pos())
        
        screen.fill((255, 255, 255))  # Fill the screen with white
        street.draw(screen)
        pygame.display.flip()
        clock.tick(60)

    pygame.quit()
    sys.exit()

if __name__ == "__main__":
    main()


pygame 2.1.2 (SDL 2.0.16, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


AttributeError: 'Street' object has no attribute 'handle_events'

: 

In [5]:
import pygame
import queue
from enum import Enum

class Direction(Enum):
    UP = "up"
    DOWN = "down"
    PLACE = "place"

class Elevator:
    def __init__(self, number, height, width, height_screen, x):
        self.image = pygame.image.load("elv.png")
        self.image = pygame.transform.scale(self.image, (width, height))
        self.ring = pygame.mixer.music.load("ding.mp3")
        self.number = number
        self.floor = 0
        self.location = (x, height_screen - self.image.get_rect().height)
        self.target_floor = 0
        self.floor_height = height
        self.elevator_speed = 0.5  # Floors per second
        self.time_elapsed = 0
        self.stay_time = 0
        self.queue = queue.Queue()
        self.last_floor = 0
        self.travel_direction = Direction.PLACE
        self.stay = False

    def draw(self, surface):
        surface.blit(self.image, self.location)

    def update_position(self, current_time, last_time, height_floor):
        distance_to_move = (current_time - last_time) * self.floor_height / self.elevator_speed
        if self.travel_direction == Direction.UP and self.location[1] > height_floor:
            self.location = (self.location[0], self.location[1] - distance_to_move)
        elif self.travel_direction == Direction.DOWN and self.location[1] < height_floor:
            self.location = (self.location[0], self.location[1] + distance_to_move)
        else:
            self.travel_direction = Direction.PLACE
            self.stay = True
            return 1
        return 0

    def adjust_stay_time(self, current_time, last_time):
        self.stay_time += current_time - last_time
        if self.stay_time >= 2:
            self.stay = False
            self.stay_time = 0

    def process_movement(self, height_floor, current_time, last_time):
        if self.time_elapsed > 0:
            self.add_time(last_time - current_time)
        else:
            self.time_elapsed = 0
        if self.stay:
            self.adjust_stay_time(current_time, last_time)
        elif self.travel_direction != Direction.PLACE:
            if self.update_position(current_time, last_time, height_floor):
                pygame.mixer.music.play()
                return self.target_floor
        elif not self.queue.empty():
            self.floor = self.target_floor
            self.target_floor = self.queue.get()
            if self.floor < self.target_floor:
                self.travel_direction = Direction.UP
            else:
                self.travel_direction = Direction.DOWN
        return -1

    def add_to_queue(self, number):
        self.queue.put(number)
        time_to_add = abs(number - self.last_floor) * self.elevator_speed + 2
        self.last_floor = number
        self.add_time(time_to_add)

    def add_time(self, number):
        self.time_elapsed += number

    def calculate_time(self, floor):
        return self.time_elapsed + abs(floor - self.last_floor) / 2

class Floor:
    def __init__(self, number, x, y, width, height):
        self.number = number
        self.rect = pygame.Rect(x, y, width, height)
        self.color = (192, 192, 192)
        self.brick_color = (255, 0, 0)
        self.black_line_color = (0, 0, 0)
        self.height_black_line = 4
        self.font = pygame.font.SysFont(None, 24)
        self.number_color = (0, 0, 0)
        self.time = 0

    def draw(self, surface):
        self.draw_bricks(surface)
        self.draw_black_line(surface)
        self.draw_number(surface)
        if self.time > 0:
            self.draw_timer(surface)

    def draw_bricks(self, surface):
        pygame.draw.rect(surface, self.color, self.rect)
        brick_width = 4
        brick_height = 2
        spacing = 1
        i = 0
        for row in range(0, self.rect.height - brick_height, brick_height + spacing):
            # Adjust col start based on the row index to create a staggered brick pattern
            col_start = int(brick_width / 2) if i % 2 == 0 else 0
            for col in range(col_start, self.rect.width - brick_width, brick_width + spacing):
                brick_rect = pygame.Rect(self.rect.left + col, self.rect.top + row, brick_width, brick_height)
                pygame.draw.rect(surface, self.brick_color, brick_rect)
            i += 1

    def draw_black_line(self, surface):
        line_rect = pygame.Rect(self.rect.left, self.rect.bottom - self.height_black_line, 
                                self.rect.width, self.height_black_line)
        pygame.draw.rect(surface, self.black_line_color, line_rect)

    def draw_number(self, surface):
        # Calculate position and color settings
        number_rect_width, number_rect_height = 20, 16
        number_rect_position, self.number_color = self.calculate_number_position(number_rect_width, number_rect_height)

        # Draw the background rectangle for the number
        pygame.draw.rect(surface, (192, 192, 192), number_rect_position)
        
        # Render the number text and blit to the surface
        number_text = self.font.render(str(self.number), True, self.number_color)
        text_rect = number_text.get_rect(center=number_rect_position.center)
        surface.blit(number_text, text_rect)

    def calculate_number_position(self, width, height):
        # Determine color and position based on the timer state
        if self.time > 0:
            color = (0, 255, 0)
            position = pygame.Rect(self.rect.right - width - 10,
                                   self.rect.centery - (height / 2),
                                   width, height)
        else:
            color = (0, 0, 0)
            position = pygame.Rect(self.rect.centerx - (width / 2),
                                   self.rect.centery - (height / 2),
                                   width, height)
        return position, color
    
    def draw_timer(self, surface):
        timer_text = self.font.render(f"{self.time:.1f}", True, (255, 255, 255))
        timer_rect = timer_text.get_rect(center=(self.rect.left + 20, self.rect.centery))
        surface.blit(timer_text, timer_rect)

    def handle_events(self, mouse_pos):
        if self.rect.collidepoint(mouse_pos) and self.time <= 0:
            return self.number
        return -1

    def get_rect(self):
        return self.rect

    def timer(self, current_time, last_time):
        if self.time > 0:
            self.time -= current_time-last_time
        else:
            self.time = 0

    def increment_timer(self, time_to_add):
        if self.time < 0:
            self.time = time_to_add
        else:
            self.time += time_to_add

class Building:
    def __init__(self, position, num_floors, num_elevators, floor_width, floor_height, height_screen):        
        self.last_time = pygame.time.get_ticks() / 1000 #in second
        self.num_floors = num_floors
        self.floor_width = floor_width
        self.floor_height = floor_height
        self.elevators = []
        for i in range(num_elevators):
            elv = Elevator(i, floor_height, floor_width/2 , height_screen, floor_width * (i/2 +1) + position)
            self.elevators.append(elv)
        self.floors = []
        for i in range(num_floors+1):
            floor = Floor(i, position, height_screen - (i + 1) * floor_height, floor_width, floor_height)
            self.floors.append(floor)

    def draw(self, surface):
        for floor in self.floors:
            floor.draw(surface)
        for elv in self.elevators:
            elv.draw(surface)

    def handle_events(self, mouse_pos):
        for floor in self.floors:
            check = floor.handle_events(mouse_pos)
            if check == floor.number:
                min_time = float('inf')
                min_elv = 0
                for elv in self.elevators:
                    time_elv = elv.calculate_time(check)
                    if elv.target_floor == check:
                        return
                    if time_elv < min_time:
                        min_time = time_elv
                        min_elv = elv.number

                self.elevators[min_elv].add_to_queue(check)
                self.floors[check].increment_timer(min_time)
                return
                        
    def process_elevator_movement(self):
        current_time = pygame.time.get_ticks() / 1000 #in second
        for elv in self.elevators:
            tarrget_floor = elv.target_floor
            height_floor = self.floors[tarrget_floor].get_rect().top
            floor = elv.process_movement(height_floor, current_time, self.last_time)
        for floor in self.floors:
            floor.timer(current_time, self.last_time)
        self.last_time = current_time

class Street:
    def __init__(self, buidings_info, height_screen, width_screen):
        self.buildings = []
        sum = 0
        max_floor = 0
        for i in buidings_info:
            sum += i[1]/2 + 1
            if i[0] > max_floor:
                max_floor = i[0]
        max_floor += 1
        count = 0
        for i in buidings_info:
            position = width_screen/sum * count + 10
            buiding = Building(position, i[0], i[1], (width_screen - 20)/sum, (height_screen-10)/max_floor, height_screen)
            self.buildings.append(buiding)
            count += i[1]/2 + 1

    def draw(self, surface):
        for building in self.buildings:
            building.draw(surface)
            building.process_elevator_movement()
            
    def handle_events(self, mouse_pos):
        for building in self.buildings:
            building.handle_events(mouse_pos)

def main():
    height_screen = 900
    width_screen = 1700
    buildings_info = [
        [15, 3],  # building1: 15 floors, 3 elevators
        [10, 2],  # building2: 10 floors, 2 elevators
        [20, 4]   # building3: 20 floors, 4 elevators
    ]
    
    pygame.init()
    screen = pygame.display.set_mode((width_screen, height_screen))
    pygame.display.set_caption("Building Floors")
    street = Street(buildings_info, height_screen, width_screen)
    clock = pygame.time.Clock()

    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
                street.handle_events(pygame.mouse.get_pos())
        
        screen.fill((255, 255, 255))  # Fill the screen with white
        street.draw(screen)
        pygame.display.flip()
        clock.tick(60)

    pygame.quit()
    sys.exit()

if __name__ == "__main__":
    main()


SystemExit: 

In [7]:
import pygame
import queue
from enum import Enum

class Direction(Enum):
    UP = "up"
    DOWN = "down"
    PLACE = "place"

class Elevator:
    def __init__(self, number, height, width, height_screen, x):
        self.image = pygame.image.load("elv.png")
        self.image = pygame.transform.scale(self.image, (width, height))
        self.ring = pygame.mixer.music.load("ding.mp3")
        self.number = number
        self.floor = 0
        self.location = (x, height_screen - self.image.get_rect().height)
        self.target_floor = 0
        self.floor_height = height
        self.elevator_speed = 0.5  # Floors per second
        self.time_elapsed = 0
        self.stay_time = 0
        self.queue = queue.Queue()
        self.last_floor = 0
        self.travel_direction = Direction.PLACE
        self.stay = False

    def draw(self, surface):
        surface.blit(self.image, self.location)

    def update_position(self, current_time, last_time, height_floor):
        distance_to_move = (current_time - last_time) * self.floor_height / self.elevator_speed
        if self.travel_direction == Direction.UP and self.location[1] > height_floor:
            self.location = (self.location[0], self.location[1] - distance_to_move)
        elif self.travel_direction == Direction.DOWN and self.location[1] < height_floor:
            self.location = (self.location[0], self.location[1] + distance_to_move)
        else:
            self.travel_direction = Direction.PLACE
            self.stay = True
            return 1
        return 0

    def adjust_stay_time(self, current_time, last_time):
        self.stay_time += current_time - last_time
        if self.stay_time >= 2:
            self.stay = False
            self.stay_time = 0

    def process_movement(self, height_floor, current_time, last_time):
        if self.time_elapsed > 0:
            self.add_time(last_time - current_time)
        else:
            self.time_elapsed = 0
        if self.stay:
            self.adjust_stay_time(current_time, last_time)
        elif self.travel_direction != Direction.PLACE:
            if self.update_position(current_time, last_time, height_floor):
                pygame.mixer.music.play()
                return self.target_floor
        elif not self.queue.empty():
            self.floor = self.target_floor
            self.target_floor = self.queue.get()
            if self.floor < self.target_floor:
                self.travel_direction = Direction.UP
            else:
                self.travel_direction = Direction.DOWN
        return -1

    def add_to_queue(self, number):
        self.queue.put(number)
        time_to_add = abs(number - self.last_floor) * self.elevator_speed + 2
        self.last_floor = number
        self.add_time(time_to_add)

    def add_time(self, number):
        self.time_elapsed += number

    def calculate_time(self, floor):
        return self.time_elapsed + abs(floor - self.last_floor) / 2

class Floor:
    def __init__(self, number, x, y, width, height):
        self.number = number
        self.rect = pygame.Rect(x, y, width, height)
        self.color = (192, 192, 192)
        self.brick_color = (255, 0, 0)
        self.black_line_color = (0, 0, 0)
        self.height_black_line = 4
        self.font = pygame.font.SysFont(None, 24)
        self.number_color = (0, 0, 0)
        self.time = 0

    def draw(self, surface):
        self.draw_bricks(surface)
        self.draw_black_line(surface)
        self.draw_number(surface)
        if self.time > 0:
            self.draw_timer(surface)

    def draw_bricks(self, surface):
        pygame.draw.rect(surface, self.color, self.rect)
        brick_width = 4
        brick_height = 2
        spacing = 1
        i = 0
        for row in range(0, self.rect.height - brick_height, brick_height + spacing):
            # Adjust col start based on the row index to create a staggered brick pattern
            col_start = int(brick_width / 2) if i % 2 == 0 else 0
            for col in range(col_start, self.rect.width - brick_width, brick_width + spacing):
                brick_rect = pygame.Rect(self.rect.left + col, self.rect.top + row, brick_width, brick_height)
                pygame.draw.rect(surface, self.brick_color, brick_rect)
            i += 1

    def draw_black_line(self, surface):
        line_rect = pygame.Rect(self.rect.left, self.rect.bottom - self.height_black_line, 
                                self.rect.width, self.height_black_line)
        pygame.draw.rect(surface, self.black_line_color, line_rect)

    def draw_number(self, surface):
        # Calculate position and color settings
        number_rect_width, number_rect_height = 20, 16
        number_rect_position, self.number_color = self.calculate_number_position(number_rect_width, number_rect_height)

        # Draw the background rectangle for the number
        pygame.draw.rect(surface, (192, 192, 192), number_rect_position)
        
        # Render the number text and blit to the surface
        number_text = self.font.render(str(self.number), True, self.number_color)
        text_rect = number_text.get_rect(center=number_rect_position.center)
        surface.blit(number_text, text_rect)

    def calculate_number_position(self, width, height):
        # Determine color and position based on the timer state
        if self.time > 0:
            color = (0, 255, 0)
            position = pygame.Rect(self.rect.right - width - 10,
                                   self.rect.centery - (height / 2),
                                   width, height)
        else:
            color = (0, 0, 0)
            position = pygame.Rect(self.rect.centerx - (width / 2),
                                   self.rect.centery - (height / 2),
                                   width, height)
        return position, color
    
    def draw_timer(self, surface):
        timer_text = self.font.render(f"{self.time:.1f}", True, (255, 255, 255))
        timer_rect = timer_text.get_rect(center=(self.rect.left + 20, self.rect.centery))
        surface.blit(timer_text, timer_rect)

    def handle_events(self, mouse_pos):
        if self.rect.collidepoint(mouse_pos) and self.time <= 0:
            return self.number
        return -1

    def get_rect(self):
        return self.rect

    def timer(self, current_time, last_time):
        if self.time > 0:
            self.time -= current_time-last_time
        else:
            self.time = 0

    def increment_timer(self, time_to_add):
        if self.time < 0:
            self.time = time_to_add
        else:
            self.time += time_to_add

class Building:
    def __init__(self, position, num_floors, num_elevators, floor_width, floor_height, height_screen):        
        self.last_time = pygame.time.get_ticks() / 1000 #in second
        self.num_floors = num_floors
        self.floor_width = floor_width
        self.floor_height = floor_height
        self.elevators = []
        for i in range(num_elevators):
            elv = Elevator(i, floor_height, floor_width/2 , height_screen, floor_width * (i/2 +1) + position)
            self.elevators.append(elv)
        self.floors = []
        for i in range(num_floors+1):
            floor = Floor(i, position, height_screen - (i + 1) * floor_height, floor_width, floor_height)
            self.floors.append(floor)

    def draw(self, surface):
        for floor in self.floors:
            floor.draw(surface)
        for elv in self.elevators:
            elv.draw(surface)

    def handle_events(self, mouse_pos):
        for floor in self.floors:
            check = floor.handle_events(mouse_pos)
            if check == floor.number:
                min_time = float('inf')
                min_elv = 0
                for elv in self.elevators:
                    time_elv = elv.calculate_time(check)
                    if elv.target_floor == check:
                        return
                    if time_elv < min_time:
                        min_time = time_elv
                        min_elv = elv.number

                self.elevators[min_elv].add_to_queue(check)
                self.floors[check].increment_timer(min_time)
                return
                        
    def process_elevator_movement(self):
        current_time = pygame.time.get_ticks() / 1000 #in second
        for elv in self.elevators:
            tarrget_floor = elv.target_floor
            height_floor = self.floors[tarrget_floor].get_rect().top
            floor = elv.process_movement(height_floor, current_time, self.last_time)
        for floor in self.floors:
            floor.timer(current_time, self.last_time)
        self.last_time = current_time


class BuildingSimulation:
    def __init__(self, buidings_info, height_screen, width_screen):
        self.buildings = []
        sum = 0
        max_floor = 0
        for i in buidings_info:
            sum += i[1]/2 + 1
            if i[0] > max_floor:
                max_floor = i[0]
        max_floor += 1
        count = 0
        for i in buidings_info:
            position = width_screen/sum * count + 10
            buiding = Building(position, i[0], i[1], (width_screen - 20)/sum, (height_screen-10)/max_floor, height_screen)
            self.buildings.append(buiding)
            count += i[1]/2 + 1

    def draw(self, surface):
        for building in self.buildings:
            building.draw(surface)
            building.process_elevator_movement()
            
    def handle_events(self, mouse_pos):
        for building in self.buildings:
            building.handle_events(mouse_pos)

    def run(self):
        height_screen = 900
        width_screen = 1700
        buildings_info = [
            [15, 3],  # building1: 15 floors, 3 elevators
            [10, 2],  # building2: 10 floors, 2 elevators
            [20, 4]   # building3: 20 floors, 4 elevators
        ]
        
        pygame.init()
        screen = pygame.display.set_mode((width_screen, height_screen))
        pygame.display.set_caption("Building Floors")
        clock = pygame.time.Clock()

        running = True
        while running:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False
                elif event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
                    self.handle_events(pygame.mouse.get_pos())
            
            screen.fill((255, 255, 255))  # Fill the screen with white
            self.draw(screen)
            pygame.display.flip()
            clock.tick(60)

        pygame.quit()
        sys.exit()

def main():
    pygame.init()
    pygame.mixer.init()  # Inicializar el mezclador de sonido

    buildings_info = [
        [15, 3],  # building1: 15 floors, 3 elevators
        [10, 2],  # building2: 10 floors, 2 elevators
        [20, 4]   # building3: 20 floors, 4 elevators
    ]
    
    simulation = BuildingSimulation(buildings_info, 900, 1700)
    simulation.run()

if __name__ == "__main__":
    main()




SystemExit: 

In [9]:
import pygame
import queue
from enum import Enum

class Direction(Enum):
    UP = "up"
    DOWN = "down"
    PLACE = "place"

class Elevator:
    def __init__(self, number, height, width, height_screen, x):
        self.image = pygame.image.load("elv.png")
        self.image = pygame.transform.scale(self.image, (width, height))
        self.ring = pygame.mixer.music.load("ding.mp3")
        self.number = number
        self.floor = 0
        self.location = (x, height_screen - self.image.get_rect().height)
        self.target_floor = 0
        self.floor_height = height
        self.elevator_speed = 0.5  # Floors per second
        self.time_elapsed = 0
        self.stay_time = 0
        self.queue = queue.Queue()
        self.last_floor = 0
        self.travel_direction = Direction.PLACE
        self.stay = False

    def draw(self, surface):
        surface.blit(self.image, self.location)

    def update_position(self, current_time, last_time, height_floor):
        distance_to_move = (current_time - last_time) * self.floor_height / self.elevator_speed
        if self.travel_direction == Direction.UP and self.location[1] > height_floor:
            self.location = (self.location[0], self.location[1] - distance_to_move)
        elif self.travel_direction == Direction.DOWN and self.location[1] < height_floor:
            self.location = (self.location[0], self.location[1] + distance_to_move)
        else:
            self.travel_direction = Direction.PLACE
            self.stay = True
            return 1
        return 0

    def adjust_stay_time(self, current_time, last_time):
        self.stay_time += current_time - last_time
        if self.stay_time >= 2:
            self.stay = False
            self.stay_time = 0

    def process_movement(self, height_floor, current_time, last_time):
        if self.time_elapsed > 0:
            self.add_time(last_time - current_time)
        else:
            self.time_elapsed = 0
        if self.stay:
            self.adjust_stay_time(current_time, last_time)
        elif self.travel_direction != Direction.PLACE:
            if self.update_position(current_time, last_time, height_floor):
                pygame.mixer.music.play()
                return self.target_floor
        elif not self.queue.empty():
            self.floor = self.target_floor
            self.target_floor = self.queue.get()
            if self.floor < self.target_floor:
                self.travel_direction = Direction.UP
            else:
                self.travel_direction = Direction.DOWN
        return -1

    def add_to_queue(self, number):
        self.queue.put(number)
        time_to_add = abs(number - self.last_floor) * self.elevator_speed + 2
        self.last_floor = number
        self.add_time(time_to_add)

    def add_time(self, number):
        self.time_elapsed += number

    def calculate_time(self, floor):
        return self.time_elapsed + abs(floor - self.last_floor) / 2

class Floor:
    def __init__(self, number, x, y, width, height):
        self.number = number
        self.rect = pygame.Rect(x, y, width, height)
        self.color = (192, 192, 192)
        self.brick_color = (255, 0, 0)
        self.black_line_color = (0, 0, 0)
        self.height_black_line = 4
        self.font = pygame.font.SysFont(None, 24)
        self.number_color = (0, 0, 0)
        self.time = 0

    def draw(self, surface):
        self.draw_bricks(surface)
        self.draw_black_line(surface)
        self.draw_number(surface)
        if self.time > 0:
            self.draw_timer(surface)

    def draw_bricks(self, surface):
        pygame.draw.rect(surface, self.color, self.rect)
        brick_width = 4
        brick_height = 2
        spacing = 1
        i = 0
        for row in range(0, self.rect.height - brick_height, brick_height + spacing):
            # Adjust col start based on the row index to create a staggered brick pattern
            col_start = int(brick_width / 2) if i % 2 == 0 else 0
            for col in range(col_start, self.rect.width - brick_width, brick_width + spacing):
                brick_rect = pygame.Rect(self.rect.left + col, self.rect.top + row, brick_width, brick_height)
                pygame.draw.rect(surface, self.brick_color, brick_rect)
            i += 1

    def draw_black_line(self, surface):
        line_rect = pygame.Rect(self.rect.left, self.rect.bottom - self.height_black_line, 
                                self.rect.width, self.height_black_line)
        pygame.draw.rect(surface, self.black_line_color, line_rect)

    def draw_number(self, surface):
        # Calculate position and color settings
        number_rect_width, number_rect_height = 20, 16
        number_rect_position, self.number_color = self.calculate_number_position(number_rect_width, number_rect_height)

        # Draw the background rectangle for the number
        pygame.draw.rect(surface, (192, 192, 192), number_rect_position)
        
        # Render the number text and blit to the surface
        number_text = self.font.render(str(self.number), True, self.number_color)
        text_rect = number_text.get_rect(center=number_rect_position.center)
        surface.blit(number_text, text_rect)

    def calculate_number_position(self, width, height):
        # Determine color and position based on the timer state
        if self.time > 0:
            color = (0, 255, 0)
            position = pygame.Rect(self.rect.right - width - 10,
                                   self.rect.centery - (height / 2),
                                   width, height)
        else:
            color = (0, 0, 0)
            position = pygame.Rect(self.rect.centerx - (width / 2),
                                   self.rect.centery - (height / 2),
                                   width, height)
        return position, color
    
    def draw_timer(self, surface):
        timer_text = self.font.render(f"{self.time:.1f}", True, (255, 255, 255))
        timer_rect = timer_text.get_rect(center=(self.rect.left + 20, self.rect.centery))
        surface.blit(timer_text, timer_rect)

    def handle_events(self, mouse_pos):
        if self.rect.collidepoint(mouse_pos) and self.time <= 0:
            return self.number
        return -1

    def get_rect(self):
        return self.rect

    def timer(self, current_time, last_time):
        if self.time > 0:
            self.time -= current_time-last_time
        else:
            self.time = 0

    def increment_timer(self, time_to_add):
        if self.time < 0:
            self.time = time_to_add
        else:
            self.time += time_to_add

class Building:
    def __init__(self, position, num_floors, num_elevators, floor_width, floor_height, height_screen):        
        self.last_time = pygame.time.get_ticks() / 1000 #in second
        self.num_floors = num_floors
        self.floor_width = floor_width
        self.floor_height = floor_height
        self.elevators = []
        for i in range(num_elevators):
            elv = Elevator(i, floor_height, floor_width/2 , height_screen, floor_width * (i/2 +1) + position)
            self.elevators.append(elv)
        self.floors = []
        for i in range(num_floors+1):
            floor = Floor(i, position, height_screen - (i + 1) * floor_height, floor_width, floor_height)
            self.floors.append(floor)

    def draw(self, surface):
        for floor in self.floors:
            floor.draw(surface)
        for elv in self.elevators:
            elv.draw(surface)

    def handle_events(self, mouse_pos):
        for floor in self.floors:
            check = floor.handle_events(mouse_pos)
            if check == floor.number:
                min_time = float('inf')
                min_elv = 0
                for elv in self.elevators:
                    time_elv = elv.calculate_time(check)
                    if elv.target_floor == check:
                        return
                    if time_elv < min_time:
                        min_time = time_elv
                        min_elv = elv.number

                self.elevators[min_elv].add_to_queue(check)
                self.floors[check].increment_timer(min_time)
                return
                        
    def process_elevator_movement(self):
        current_time = pygame.time.get_ticks() / 1000 #in second
        for elv in self.elevators:
            tarrget_floor = elv.target_floor
            height_floor = self.floors[tarrget_floor].get_rect().top
            floor = elv.process_movement(height_floor, current_time, self.last_time)
        for floor in self.floors:
            floor.timer(current_time, self.last_time)
        self.last_time = current_time


def factory(object_type, *args, **kwargs):
    type_dict = {
        "building": Building,
        "floor": Floor,
        "elevator": Elevator
    }
    return type_dict[object_type](*args, **kwargs) if object_type in type_dict else None


class BuildingSimulation:
    def __init__(self, buidings_info, height_screen, width_screen):
        self.buildings = []
        sum = 0
        max_floor = 0
        for i in buidings_info:
            sum += i[1]/2 + 1
            if i[0] > max_floor:
                max_floor = i[0]
        max_floor += 1
        count = 0
        for i in buidings_info:
            position = width_screen/sum * count + 10
            building = Building(position, i[0], i[1], (width_screen - 20)/sum, (height_screen-10)/max_floor, height_screen)
            self.buildings.append(building)
            count += i[1]/2 + 1

    def draw(self, surface):
        for building in self.buildings:
            building.draw(surface)
            building.process_elevator_movement()
            
    def handle_events(self, mouse_pos):
        for building in self.buildings:
            building.handle_events(mouse_pos)

    def run(self):
        height_screen = 900
        width_screen = 1700
        buildings_info = [
            [15, 3],  # building1: 15 floors, 3 elevators
            [10, 2],  # building2: 10 floors, 2 elevators
            [20, 4]   # building3: 20 floors, 4 elevators
        ]
        
        pygame.init()
        screen = pygame.display.set_mode((width_screen, height_screen))
        pygame.display.set_caption("Building Floors")
        clock = pygame.time.Clock()

        running = True
        while running:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False
                elif event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
                    self.handle_events(pygame.mouse.get_pos())
            
            screen.fill((255, 255, 255))  # Fill the screen with white
            self.draw(screen)
            pygame.display.flip()
            clock.tick(60)

        pygame.quit()
        sys.exit()

def main():
    pygame.init()
    pygame.mixer.init()  # Inicializar el mezclador de sonido

    buildings_info = [
        [15, 3],  # building1: 15 floors, 3 elevators
        [10, 2],  # building2: 10 floors, 2 elevators
        [20, 4]   # building3: 20 floors, 4 elevators
    ]
    
    simulation = BuildingSimulation(buildings_info, 900, 1700)
    simulation.run()

if __name__ == "__main__":
    main()




SystemExit: 